# DecisionTree Assignment - 20기 OOO

물음표 친 부분을 채우고 코드에 대한 주석을 자세하게 달아주세요!

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [50]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## Gini 계수를 구하는 함수 만들기

<img src="gini.png" width="200">

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

- 지니계수는 데이터의 통계적 분산 정도를 정량화 해서 표현한 값이다.
- 어떤 집합의 gini index가 높을수록 그 집단의 데이터가 분산되어 있음을 확인할 수 있다.

In [3]:
def get_gini(df, label):
    # calculate the gini index of a dataset
    # df: dataframe
    # label: the label column name
    # return: gini index
    gini = 1
    total = len(df)
    for i in df[label].unique():
        gini -= (len(df[df[label] == i]) / total) ** 2
    
        
    
    
    return gini

In [27]:
pd_data['class_buys_computer']

0      no
1      no
2     yes
3     yes
4     yes
5      no
6     yes
7      no
8     yes
9     yes
10    yes
11    yes
12    yes
13     no
Name: class_buys_computer, dtype: object

In [4]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [14]:
from itertools import combinations

def get_binary_split(df, attribute):
    
    uniques = list(df[attribute].unique()) # 속성 데이터 고유값들을 담은 리스트 
    uniques.sort() # 고유값들을 오름차순으로 정렬
    results=[]
    for i in range(len(df)):
        if i == 0:
            continue
        splits = list(combinations(uniques, i)) # 고유값들을 2개씩 묶어서 조합을 만듦
        for split in splits:
            results.append(split)



    
    return results

In [15]:
get_binary_split(pd_data,'income')

[('high',),
 ('low',),
 ('medium',),
 ('high', 'low'),
 ('high', 'medium'),
 ('low', 'medium'),
 ('high', 'low', 'medium')]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [75]:
def get_attribute_gini_index(df, attribute, label):
    
    result = {}
    binary_split = get_binary_split(df, attribute)
    n = len(df)
    # get the gini index for each split
    for split in binary_split:
        # get the gini index for each label
        # if df['attribute'] == split, then the label is 1, otherwise 0
        # get the gini index for each label
        a=copy.deepcopy(df)
        a[attribute] = a[attribute].apply(lambda x: 1 if x in split else 0)
        left = a[a[attribute] == 1]
        right = a[a[attribute] == 0]
        leftgini=get_gini(left,label)
        rightgini=get_gini(right,label)
        result[split] = (len(left)/n)*leftgini + (len(right)/n)*rightgini



    
    return result

In [76]:
get_attribute_gini_index(pd_data, 'income', 'class_buys_computer')

{('high',): 0.4428571428571429,
 ('low',): 0.45,
 ('medium',): 0.4583333333333333,
 ('high', 'low'): 0.4583333333333333,
 ('high', 'medium'): 0.45,
 ('low', 'medium'): 0.4428571428571429,
 ('high', 'low', 'medium'): 0.4591836734693877}

- 여기서 가장 작은 Gini index값을 가지는 class를 확인합니다.

In [77]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())

(('high',), 0.4428571428571429)

In [78]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())[0]

('high',)

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [79]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

In [80]:
features

['age', 'income', 'student', 'credit_rating']

In [81]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

# 각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)
for feature in features:
    #minimum in dictionary
    s=(get_attribute_gini_index(pd_data, feature, label))
    # sort by value
    s=sorted(s.items(), key=lambda x: x[1])
    print(s[0])
    

(('middle_aged',), 0.35714285714285715)
(('high',), 0.4428571428571429)
(('no',), 0.3673469387755103)
(('excellent',), 0.42857142857142855)


gini index가 가장 작게 나온 'age'를 가장 중요한 변수로 선정합니다.

이어서 해당 변수의 이진 분류된 각 class에 대해 Gini index도 계산합니다.

In [82]:
get_attribute_gini_index(pd_data, 'age', 'class_buys_computer')

{('middle_aged',): 0.35714285714285715,
 ('senior',): 0.45714285714285713,
 ('youth',): 0.3936507936507937,
 ('middle_aged', 'senior'): 0.3936507936507937,
 ('middle_aged', 'youth'): 0.45714285714285713,
 ('senior', 'youth'): 0.35714285714285715,
 ('middle_aged', 'senior', 'youth'): 0.4591836734693877}

'age' 변수에서 gini index가 가장 작게 나온 'middle_aged' class를 선정합니다.

## Entropy 를 구하는 함수 만들기

<img src = https://miro.medium.com/max/1122/0*DkWdyGidNSfdT1Nu.png width = "350">

In [83]:
from math import log2

def getEntropy(df, feature) :
    entropy = 0
    total = len(df)
    for i in df[feature].unique() :
        count = len(df[df[feature] == i])
        entropy += -1 * (count/total) * log2(count/total)
    return entropy


In [84]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706311

In [92]:
# 가장 중요한 변수로 선정된 목표변수를 제외한 다른 변수들에 대해
# 각 칼럼별로 엔트로피를 구해주는 함수를 작성해주세요.
from collections import defaultdict

def getGainA(df, feature) :
    # feature : 목표변수
    # df : 데이터프레임
    # return : dictionary for each feature
    #           key : feature name
    #           value : entropy
    a=defaultdict(float)
    for col in df.columns:
        if col == feature:
            continue
        else:
            # save in dictionary
            a[col]=(getEntropy(df, col))
    return a



In [93]:
getGainA(pd_data, "class_buys_computer")

defaultdict(float,
            {'age': 1.5774062828523454,
             'income': 1.5566567074628228,
             'student': 1.0,
             'credit_rating': 0.9852281360342516})